# Add Geographical places to existing events

In [1]:
# %load /Users/gaetanmuck/Development/geovpylib/templates/heading-admin.py
%load_ext autoreload
%autoreload 2

# Common imports
import os
import pandas as pd, numpy as np
import datetime
# import math
#import time
#import json
#import requests
#import duckdb
#import plotly.express as px
# from multiprocessing import Pool

# Geovpylib library
import geovpylib.analysis as a
import geovpylib.database as db
import geovpylib.decorators as d
import geovpylib.importer as i
import geovpylib.magics
import geovpylib.pks as pks
import geovpylib.queries as q
import geovpylib.record_linkage as rl
import geovpylib.sparql as sparql
import geovpylib.utils as u
eta = u.Eta()

# Specific imports
# ...

# Global variables
# ...

# Connect to Geovistory database for insert
env = 'prod' # Database to query: "prod", "stag", "dev", "local"
pk_project = pks.projects.symogih # The project to query/insert: integer
execute = True # Boolean to prevent to execute directly into databases
metadata_str = '' # kebab-lower-case or snake-lower-case. 
import_manner = 'one-shot' # 'one-shot' or 'batch'


# Get matching BHP <=> GV

In [2]:
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

matchings_pks = db.query(f"""
    select 
        s1.fk_subject_info as pk_entity_gv,
        a3.string as uri
    from information.resource r
    inner join projects.info_proj_rel ipr on ipr.fk_entity = r.pk_entity and ipr.fk_project = {pks.projects.symogih} and ipr.is_in_project = true
    inner join information.statement s1 on s1.fk_object_info = r.pk_entity and s1.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pks.projects.symogih} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = s1.fk_object_info and s2.fk_property = {pks.properties.uri_hasValue_appellation}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pks.projects.symogih} and ipr2.is_in_project = true
    inner join information.appellation a3 on a3.pk_entity = s2.fk_object_info
    where r.fk_class = {pks.classes.uri}
""")
matchings_pks = matchings_pks[matchings_pks['uri'].str.contains('symogih.org')]
matchings_pks['pk_entity_bhp'] = [text[text.rindex('/') + 1:] for text in matchings_pks['uri']]
# matchings_pks = matchings_pks[['pk_entity_gv', 'pk_entity_bhp']]

matchings_pks

[DB] Connecting to PRODUCTION Database ... Connected!


,pk_entity_gv,uri,pk_entity_bhp
196,732700,http://symogih.org/resource/CoAc13168,CoAc13168
198,6140213,http://symogih.org/resource/CoAc13701,CoAc13701
199,6140228,http://symogih.org/resource/CoAc13724,CoAc13724
203,6140243,http://symogih.org/resource/CoAc11559,CoAc11559
204,6140258,http://symogih.org/resource/CoAc13727,CoAc13727
...,...,...,...
262399,11146348,http://symogih.org/resource/NaPl126841,NaPl126841
262400,11146349,http://symogih.org/resource/NaPl126845,NaPl126845
262401,11146350,http://symogih.org/resource/NaPl126846,NaPl126846
262402,11146351,http://symogih.org/resource/NaPl126851,NaPl126851


# Set geographical places to births

**Get infos from BHP**

In [3]:
db_url_env_var_name = 'YELLOW_BHP' # Name of an environment variable holding the Postgres database URL
db.connect_external(os.getenv(db_url_env_var_name), execute=execute)

# Fetch info from bhp
births_places_bhp = db.query("""
    select 
        ir2.fk_associated_object as pk_person_bhp,
        i.pk_information as pk_birth_bhp,
        ir.fk_associated_object as pk_geoplace_bhp
    from bhp.information i
    inner join bhp.information_role ir on ir.fk_information = i.pk_information and ir.fk_type_role = 8
    inner join bhp.information_role ir2 on ir2.fk_information = i.pk_information and ir2.fk_type_role = 40
    where i.fk_type_information = 14
""")
births_places_bhp['pk_birth_bhp'] = 'Info' + births_places_bhp['pk_birth_bhp'].astype(str)

a.infos(births_places_bhp)

[DB] Connecting to PGSQL Database ... Connected!
Shape:  (15711, 3) - extract:


,pk_person_bhp,pk_birth_bhp,pk_geoplace_bhp
0,Actr43963,Info87245,NaPl78319
1,Actr43871,Info87255,NaPl2488
2,Actr43740,Info87257,NaPl51757
3,Actr44082,Info87284,NaPl2193
4,Actr42382,Info87247,NaPl2170


**Match to GV entities**

In [4]:
births_places = births_places_bhp.merge(matchings_pks, left_on='pk_person_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
births_places.rename(columns={'pk_entity_gv':'pk_person_gv'}, inplace=True)
births_places = births_places.merge(matchings_pks, left_on='pk_birth_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
births_places.rename(columns={'pk_entity_gv':'pk_birth_gv'}, inplace=True)
births_places = births_places.merge(matchings_pks, left_on='pk_geoplace_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
births_places.rename(columns={'pk_entity_gv':'pk_geoplace_gv'}, inplace=True)

births_places = births_places[['pk_birth_bhp', 'pk_person_gv', 'pk_birth_gv', 'pk_geoplace_gv']]

births_places['pk_person_gv'] = births_places['pk_person_gv'].astype(pd.Int64Dtype())
births_places['pk_birth_gv'] = births_places['pk_birth_gv'].astype(pd.Int64Dtype())
births_places['pk_geoplace_gv'] = births_places['pk_geoplace_gv'].astype(pd.Int64Dtype())

births_places = births_places[pd.notna(births_places['pk_geoplace_gv'])]
births_places = births_places[pd.notna(births_places['pk_person_gv'])]

a.infos(births_places)

Shape:  (15702, 4) - extract:


,pk_birth_bhp,pk_person_gv,pk_birth_gv,pk_geoplace_gv
0,Info87245,6526371,7529134,11142959
1,Info87255,6532158,7529048,11060394
2,Info87257,6532123,7528922,3151591
3,Info87284,6513573,7529245,11104110
4,Info87247,6494650,7527592,11144597


**Create informations**

In [5]:
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

# Add a place to existing events
selection = births_places[pd.notna(births_places['pk_birth_gv'])].copy()
db.statements.create(selection['pk_birth_gv'], pks.properties.period_tookPlaceOnOrWithin_phyThing, selection['pk_geoplace_gv'])

# Create event for entities that does not have one
selection = births_places[pd.isna(births_places['pk_birth_gv'])].copy()
selection['pk_birth_gv'] = db.resources.create(pks.classes.birth, len(selection))
db.statements.create(selection['pk_birth_gv'], pks.properties.period_tookPlaceOnOrWithin_phyThing, selection['pk_geoplace_gv'])
db.statements.create(selection['pk_birth_gv'], pks.properties.birth_broughtIntoLife_person, selection['pk_person_gv'])
selection['uri'] = 'http://symogih.org/resource/' + selection['pk_birth_bhp']
db.shortcuts.add_uris(selection['pk_birth_gv'], selection['uri'])

[DB] Connecting to PRODUCTION Database ... Connected!
[DB] Creating 14885 statements ... Updating metadata ... Done in [00h00m07s]
[DB] Creating info_proj_rel of 14885 entities with project <6857901> ... Done in [00h00m03s]
[DB] Creating 817 resources of class [61] ... Done in [00h00m00s]
[DB] Creating info_proj_rel of 817 entities with project <6857901> ... Done in [00h00m01s]
[DB] Creating 817 statements ... Updating metadata ... Done in [00h00m00s]
[DB] Creating info_proj_rel of 817 entities with project <6857901> ... Done in [00h00m01s]
[DB] Creating 817 statements ... Updating metadata ... Done in [00h00m00s]
[DB] Creating info_proj_rel of 817 entities with project <6857901> ... Done in [00h00m01s]
[DB] Creating 817 resources of class [967] ... Done in [00h00m00s]
[DB] Creating info_proj_rel of 817 entities with project <6857901> ... Done in [00h00m00s]
[DB] Creating 817 appellations ... Done in [00h00m01s]
[DB] Creating 817 statements ... Updating metadata ... Done in [00h00m01s]

# Set geographical places to deaths

**Get infos from BHP**

In [6]:
db_url_env_var_name = 'YELLOW_BHP' # Name of an environment variable holding the Postgres database URL
db.connect_external(os.getenv(db_url_env_var_name), execute=execute)

# Fetch info from bhp
deaths_places_bhp = db.query("""
    select 
        ir2.fk_associated_object as pk_person_bhp,
        i.pk_information as pk_death_bhp,
        ir.fk_associated_object as pk_geoplace_bhp
    from bhp.information i
    inner join bhp.information_role ir on ir.fk_information = i.pk_information and ir.fk_type_role = 8
    inner join bhp.information_role ir2 on ir2.fk_information = i.pk_information and ir2.fk_type_role = 45
    where i.fk_type_information = 26
""")
deaths_places_bhp['pk_death_bhp'] = 'Info' + deaths_places_bhp['pk_death_bhp'].astype(str)

a.infos(deaths_places_bhp)

[DB] Connecting to PGSQL Database ... Connected!
Shape:  (6299, 3) - extract:


,pk_person_bhp,pk_death_bhp,pk_geoplace_bhp
0,Actr43740,Info87258,NaPl2085
1,Actr43871,Info87259,NaPl2461
2,Actr33850,Info87377,NaPl90401
3,Actr33851,Info87378,NaPl90401
4,Actr44135,Info87392,NaPl90402


**Match to GV entities**

In [7]:
deaths_places = deaths_places_bhp.merge(matchings_pks, left_on='pk_person_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
deaths_places.rename(columns={'pk_entity_gv':'pk_person_gv'}, inplace=True)
deaths_places = deaths_places.merge(matchings_pks, left_on='pk_death_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
deaths_places.rename(columns={'pk_entity_gv':'pk_death_gv'}, inplace=True)
deaths_places = deaths_places.merge(matchings_pks, left_on='pk_geoplace_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
deaths_places.rename(columns={'pk_entity_gv':'pk_geoplace_gv'}, inplace=True)

deaths_places = deaths_places[['pk_death_bhp', 'pk_person_gv', 'pk_death_gv', 'pk_geoplace_gv']]

deaths_places['pk_person_gv'] = deaths_places['pk_person_gv'].astype(pd.Int64Dtype())
deaths_places['pk_death_gv'] = deaths_places['pk_death_gv'].astype(pd.Int64Dtype())
deaths_places['pk_geoplace_gv'] = deaths_places['pk_geoplace_gv'].astype(pd.Int64Dtype())

deaths_places = deaths_places[pd.notna(deaths_places['pk_geoplace_gv'])]
deaths_places = deaths_places[pd.notna(deaths_places['pk_person_gv'])]

a.infos(deaths_places)

Shape:  (6349, 4) - extract:


,pk_death_bhp,pk_person_gv,pk_death_gv,pk_geoplace_gv
0,Info87258,6532123,7766697,81770
1,Info87259,6532158,7766823,11079336
2,Info87377,6517061,7758883,11070398
3,Info87378,6517022,7758884,11070398
4,Info87392,6536584,7767074,11142963


In [8]:
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

# Add a place to existing events
selection = deaths_places[pd.notna(deaths_places['pk_death_gv'])].copy()
db.statements.create(selection['pk_death_gv'], pks.properties.period_tookPlaceOnOrWithin_phyThing, selection['pk_geoplace_gv'])

# Create event for entities that does not have one
selection = deaths_places[pd.isna(deaths_places['pk_death_gv'])].copy()
selection['pk_death_gv'] = db.resources.create(pks.classes.death, len(selection))
db.statements.create(selection['pk_death_gv'], pks.properties.period_tookPlaceOnOrWithin_phyThing, selection['pk_geoplace_gv'])
db.statements.create(selection['pk_death_gv'], pks.properties.death_wasDeathOf_person, selection['pk_person_gv'])
selection['uri'] = 'http://symogih.org/resource/' + selection['pk_death_bhp']
db.shortcuts.add_uris(selection['pk_death_gv'], selection['uri'])

[DB] Connecting to PRODUCTION Database ... Connected!
[DB] Creating 6347 statements ... Updating metadata ... Done in [00h00m04s]
[DB] Creating info_proj_rel of 6347 entities with project <6857901> ... Done in [00h00m01s]
[DB] Creating 2 resources of class [63] ... Done in [00h00m00s]
[DB] Creating info_proj_rel of 2 entities with project <6857901> ... Done in [00h00m00s]
[DB] Creating 2 statements ... Updating metadata ... Done in [00h00m01s]
[DB] Creating info_proj_rel of 2 entities with project <6857901> ... Done in [00h00m00s]
[DB] Creating 2 statements ... Updating metadata ... Done in [00h00m00s]
[DB] Creating info_proj_rel of 2 entities with project <6857901> ... Done in [00h00m00s]
[DB] Creating 2 resources of class [967] ... Done in [00h00m00s]
[DB] Creating info_proj_rel of 2 entities with project <6857901> ... Done in [00h00m01s]
[DB] Creating 2 appellations ... Done in [00h00m00s]
[DB] Creating 2 statements ... Updating metadata ... Done in [00h00m00s]
[DB] Creating info_pr

# Set geographical places to Formations

**Get infos from BHP**

In [9]:
db_url_env_var_name = 'YELLOW_BHP' # Name of an environment variable holding the Postgres database URL
db.connect_external(os.getenv(db_url_env_var_name), execute=execute)

# Fetch info from bhp
formations_places_bhp = db.query("""
    select 
        ir2.fk_associated_object as pk_group_bhp,
        i.pk_information as pk_formation_bhp,
        ir.fk_associated_object as pk_geoplace_bhp
    from bhp.information i
    inner join bhp.information_role ir on ir.fk_information = i.pk_information and ir.fk_type_role = 8
    inner join bhp.information_role ir2 on ir2.fk_information = i.pk_information and ir2.fk_type_role = 49
    where i.fk_type_information = 30
""")
formations_places_bhp['pk_formation_bhp'] = 'Info' + formations_places_bhp['pk_formation_bhp'].astype(str)

a.infos(formations_places_bhp)

[DB] Connecting to PGSQL Database ... Connected!
Shape:  (7256, 3) - extract:


,pk_group_bhp,pk_formation_bhp,pk_geoplace_bhp
0,CoAc13688,Info87240,NaPl2488
1,CoAc13691,Info87272,NaPl90408
2,CoAc13692,Info87260,NaPl64299
3,CoAc13696,Info87262,NaPl1670
4,CoAc13693,Info87268,NaPl1659


**Match to GV entities**

In [10]:
formations_places = formations_places_bhp.merge(matchings_pks, left_on='pk_group_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
formations_places.rename(columns={'pk_entity_gv':'pk_group_gv'}, inplace=True)
formations_places = formations_places.merge(matchings_pks, left_on='pk_formation_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
formations_places.rename(columns={'pk_entity_gv':'pk_formation_gv'}, inplace=True)
formations_places = formations_places.merge(matchings_pks, left_on='pk_geoplace_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
formations_places.rename(columns={'pk_entity_gv':'pk_geoplace_gv'}, inplace=True)

formations_places = formations_places[['pk_formation_bhp', 'pk_group_gv', 'pk_formation_gv', 'pk_geoplace_gv']]

formations_places['pk_group_gv'] = formations_places['pk_group_gv'].astype(pd.Int64Dtype())
formations_places['pk_formation_gv'] = formations_places['pk_formation_gv'].astype(pd.Int64Dtype())
formations_places['pk_geoplace_gv'] = formations_places['pk_geoplace_gv'].astype(pd.Int64Dtype())

formations_places = formations_places[pd.notna(formations_places['pk_geoplace_gv'])]
formations_places = formations_places[pd.notna(formations_places['pk_group_gv'])]

a.infos(formations_places)

Shape:  (7195, 4) - extract:


,pk_formation_bhp,pk_group_gv,pk_formation_gv,pk_geoplace_gv
0,Info87240,8369351,10132401,11060394
1,Info87272,8369355,10132403,11145172
2,Info87260,6141395,10132404,11088053
3,Info87262,6141886,10132408,11112248
4,Info87268,6141095,10132405,11056936


**Create informations**

In [11]:
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

# Add a place to existing events
selection = formations_places[pd.notna(formations_places['pk_formation_gv'])].copy()
db.statements.create(selection['pk_formation_gv'], pks.properties.period_tookPlaceOnOrWithin_phyThing, selection['pk_geoplace_gv'])

# Create event for entities that does not have one
selection = formations_places[pd.isna(formations_places['pk_formation_gv'])].copy()
selection['pk_formation_gv'] = db.resources.create(pks.classes.formation, len(selection))
db.statements.create(selection['pk_formation_gv'], pks.properties.period_tookPlaceOnOrWithin_phyThing, selection['pk_geoplace_gv'])
db.statements.create(selection['pk_formation_gv'], pks.properties.formation_hasFormed_group, selection['pk_group_gv'])
selection['uri'] = 'http://symogih.org/resource/' + selection['pk_formation_bhp']
db.shortcuts.add_uris(selection['pk_formation_gv'], selection['uri'])

[DB] Connecting to PRODUCTION Database ... Connected!
[DB] Creating 6575 statements ... Updating metadata ... Done in [00h00m03s]
[DB] Creating info_proj_rel of 6575 entities with project <6857901> ... Done in [00h00m01s]
[DB] Creating 620 resources of class [60] ... Done in [00h00m01s]
[DB] Creating info_proj_rel of 620 entities with project <6857901> ... Done in [00h00m00s]
[DB] Creating 620 statements ... Updating metadata ... Done in [00h00m00s]
[DB] Creating info_proj_rel of 620 entities with project <6857901> ... Done in [00h00m00s]
[DB] Creating 620 statements ... Updating metadata ... Done in [00h00m01s]
[DB] Creating info_proj_rel of 620 entities with project <6857901> ... Done in [00h00m00s]
[DB] Creating 620 resources of class [967] ... Done in [00h00m01s]
[DB] Creating info_proj_rel of 620 entities with project <6857901> ... Done in [00h00m00s]
[DB] Creating 620 appellations ... Done in [00h00m00s]
[DB] Creating 620 statements ... Updating metadata ... Done in [00h00m01s]
[

# Set geographical places to Dissolutions

**Get infos from BHP**

In [12]:
db_url_env_var_name = 'YELLOW_BHP' # Name of an environment variable holding the Postgres database URL
db.connect_external(os.getenv(db_url_env_var_name), execute=execute)

# Fetch info from bhp
dissolutions_places_bhp = db.query("""
    select 
        ir2.fk_associated_object as pk_group_bhp,
        i.pk_information as pk_dissolution_bhp,
        ir.fk_associated_object as pk_geoplace_bhp
    from bhp.information i
    inner join bhp.information_role ir on ir.fk_information = i.pk_information and ir.fk_type_role = 8
    inner join bhp.information_role ir2 on ir2.fk_information = i.pk_information and ir2.fk_type_role = 54
    where i.fk_type_information = 33
""")
dissolutions_places_bhp['pk_dissolution_bhp'] = 'Info' + dissolutions_places_bhp['pk_dissolution_bhp'].astype(str)

a.infos(dissolutions_places_bhp)

[DB] Connecting to PGSQL Database ... Connected!
Shape:  (18, 3) - extract:


,pk_group_bhp,pk_dissolution_bhp,pk_geoplace_bhp
0,CoAc15408,Info112695,NaPl76163
1,CoAc64,Info131463,NaPl3626
2,CoAc13684,Info1884,NaPl1998
3,CoAc18415,Info137200,ImCo807
4,CoAc17543,Info137223,ImCo710


**Match to GV entities**

In [13]:
dissolutions_places = dissolutions_places_bhp.merge(matchings_pks, left_on='pk_group_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
dissolutions_places.rename(columns={'pk_entity_gv':'pk_group_gv'}, inplace=True)
dissolutions_places = dissolutions_places.merge(matchings_pks, left_on='pk_dissolution_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
dissolutions_places.rename(columns={'pk_entity_gv':'pk_dissolution_gv'}, inplace=True)
dissolutions_places = dissolutions_places.merge(matchings_pks, left_on='pk_geoplace_bhp', right_on='pk_entity_bhp', how='left').drop(columns=['pk_entity_bhp'])
dissolutions_places.rename(columns={'pk_entity_gv':'pk_geoplace_gv'}, inplace=True)

dissolutions_places = dissolutions_places[['pk_dissolution_bhp', 'pk_group_gv', 'pk_dissolution_gv', 'pk_geoplace_gv']]

dissolutions_places['pk_group_gv'] = dissolutions_places['pk_group_gv'].astype(pd.Int64Dtype())
dissolutions_places['pk_dissolution_gv'] = dissolutions_places['pk_dissolution_gv'].astype(pd.Int64Dtype())
dissolutions_places['pk_geoplace_gv'] = dissolutions_places['pk_geoplace_gv'].astype(pd.Int64Dtype())

dissolutions_places = dissolutions_places[pd.notna(dissolutions_places['pk_geoplace_gv'])]
dissolutions_places = dissolutions_places[pd.notna(dissolutions_places['pk_group_gv'])]

a.infos(dissolutions_places)

Shape:  (4, 4) - extract:


,pk_dissolution_bhp,pk_group_gv,pk_dissolution_gv,pk_geoplace_gv
0,Info112695,8373184,10142919,11115041
1,Info131463,8368329,10148870,81495
2,Info131463,8368329,10148870,1257291
3,Info1884,8372179,10142347,778570


**Create informations**

In [14]:
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

# Add a place to existing events
selection = dissolutions_places[pd.notna(dissolutions_places['pk_dissolution_gv'])].copy()
db.statements.create(selection['pk_dissolution_gv'], pks.properties.period_tookPlaceOnOrWithin_phyThing, selection['pk_geoplace_gv'])

# Create event for entities that does not have one
selection = dissolutions_places[pd.isna(dissolutions_places['pk_dissolution_gv'])].copy()
selection['pk_dissolution_gv'] = db.resources.create(pks.classes.dissolution, len(selection))
db.statements.create(selection['pk_dissolution_gv'], pks.properties.period_tookPlaceOnOrWithin_phyThing, selection['pk_geoplace_gv'])
db.statements.create(selection['pk_dissolution_gv'], pks.properties.dissolution_dissolved_group, selection['pk_group_gv'])
selection['uri'] = 'http://symogih.org/resource/' + selection['pk_dissolution_bhp']
db.shortcuts.add_uris(selection['pk_dissolution_gv'], selection['uri'])

[DB] Connecting to PRODUCTION Database ... Connected!
[DB] Creating 4 statements ... Updating metadata ... Done in [00h00m00s]
[DB] Creating info_proj_rel of 4 entities with project <6857901> ... Done in [00h00m00s]


---

Correction has been done afterwards : 

In [19]:
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

corrections = db.query("""
    select
        s.pk_entity, s.fk_subject_info, s.fk_property, s.fk_object_info 
    from information."statement" s
    inner join information.resource r on r.pk_entity = s.fk_subject_info and (r.fk_class = 63 or r.fk_class = 60 or r.fk_class = 62)
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s.pk_entity and ipr1.fk_project = 6857901 and ipr1.is_in_project = true
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = r.pk_entity and ipr2.fk_project = 6857901 and ipr2.is_in_project = true
    where s.fk_property = 7
""")['pk_entity']
corrections = u.get_sql_ready_str(corrections)

db.execute(f"""
    update information.statement
           set fk_property = 1599
    where pk_entity in {corrections};
""")

[DB] Connecting to PRODUCTION Database ... Connected!
